# Lab 15.1: Life Insurance Integration

**Duration:** 12 minutes  
**Difficulty:** Advanced  

## Learning Objectives

In this notebook, you will:
- Create life insurance products (Term Life and Whole Life)
- Build life insurance policies with beneficiary management
- Implement cash value tracking for whole life policies
- Query and verify life insurance relationships

---

## Step 1: Connect to Neo4j

Establish connection to the Neo4j Enterprise instance and verify the database state.

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
from datetime import datetime, date
import uuid

# Connect to Neo4j Enterprise instance
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def run_query(query, parameters=None):
    with driver.session(database="insurance") as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

# Verify current database state
current_state = run_query("""
MATCH (n) 
RETURN labels(n)[0] AS node_type, count(n) AS count
ORDER BY count DESC
""")

print("Current Database State:")
for record in current_state:
    print(f"  {record['node_type']}: {record['count']} nodes")

## Step 2: Create Life Insurance Products

### Business Context: Life Insurance Product Types

**Term Life Insurance:**
- Provides coverage for a specific term (10, 15, 20, 25, or 30 years)
- Pure death benefit with no cash value accumulation
- Lower premiums compared to permanent insurance
- Requires medical underwriting with health questions
- Coverage expires at the end of the term

**Whole Life Insurance:**
- Permanent coverage for the insured's entire lifetime
- Builds cash value that grows tax-deferred
- Higher premiums but guaranteed level premiums
- Dividend options available (paid-up additions, cash, reduce premium)
- Policy loans available against cash value
- Surrender charges in early policy years

In [ ]:
# Create life insurance products and policies
life_insurance_query = """
// Create Life Insurance Products
CREATE (term_life:Product:Insurance:Life {
  id: randomUUID(),
  product_id: "PROD-LIFE-001",
  product_name: "Term Life Plus",
  product_type: "Life",
  insurance_line: "Individual Life",
  coverage_type: "Term Life",
  
  // Product details
  term_options: [10, 15, 20, 25, 30],
  min_coverage: 25000.00,
  max_coverage: 2000000.00,
  age_range: "18-70",
  medical_exam_required: true,
  
  // Underwriting
  health_questions: 15,
  simplified_issue: false,
  guaranteed_issue: false,
  
  created_at: datetime(),
  created_by: "product_management",
  version: 1
})

CREATE (whole_life:Product:Insurance:Life {
  id: randomUUID(),
  product_id: "PROD-LIFE-002",
  product_name: "Whole Life Builder",
  product_type: "Life",
  insurance_line: "Individual Life",
  coverage_type: "Whole Life",
  
  // Product details
  min_coverage: 50000.00,
  max_coverage: 1000000.00,
  age_range: "0-85",
  cash_value_growth: 0.04,
  dividend_option: true,
  
  // Features
  loan_option: true,
  paid_up_option: true,
  surrender_charges: [0.15, 0.12, 0.10, 0.08, 0.05, 0.03, 0.01, 0.00],
  
  created_at: datetime(),
  created_by: "product_management",
  version: 1
})

RETURN term_life.product_name AS term_product,
       whole_life.product_name AS whole_product
"""

product_results = run_query(life_insurance_query)
print("Life Insurance Products Created:")
for record in product_results:
    print(f"  Term Life: {record['term_product']}")
    print(f"  Whole Life: {record['whole_product']}")

## Step 3: Create Life Insurance Customers and Policies

### Business Context: Life Insurance Underwriting

**Key Underwriting Factors:**
- **Age**: Younger applicants receive better rates
- **Health Status**: Medical exam results determine risk classification
- **Smoking Status**: Non-smokers receive significantly lower premiums
- **Occupation**: Hazardous occupations may increase premiums
- **Family Medical History**: Genetic risk factors considered
- **Lifestyle Factors**: Hobbies, travel, driving record

**Risk Classifications:**
- Preferred Plus: Best health, lowest premiums
- Preferred: Excellent health, very good rates
- Standard Plus: Good health, standard rates
- Standard: Average health, standard rates
- Substandard: Rated for health conditions

In [ ]:
# Create life insurance customers and term life policy
term_life_query = """
// Create Term Life Customer
CREATE (mike:Customer:Person {
  id: randomUUID(),
  customer_id: "CUST-LIFE-001",
  first_name: "Michael",
  last_name: "Thompson",
  full_name: "Michael Thompson",
  date_of_birth: date("1985-03-15"),
  age: 40,
  gender: "Male",
  
  // Contact information
  email: "mike.thompson@email.com",
  phone: "214-555-0180",
  
  // Address
  street_address: "1847 Oak Valley Drive",
  city: "Dallas",
  state: "TX",
  postal_code: "75201",
  country: "USA",
  
  // Life insurance specific
  smoking_status: false,
  health_status: "Excellent",
  occupation: "Software Engineer",
  annual_income: 95000.00,
  
  created_at: datetime(),
  created_by: "underwriting",
  version: 1
})

// Match the Term Life product
MATCH (term_life:Product {product_id: "PROD-LIFE-001"})

// Create Term Life Policy
CREATE (term_policy:Policy:Life:Active {
  id: randomUUID(),
  policy_number: "POL-TERM-001847",
  product_type: "Life",
  policy_type: "Term Life",
  
  // Coverage details
  face_value: 500000.00,
  term_length: 20,
  premium_mode: "Annual",
  annual_premium: 485.00,
  
  // Policy dates
  effective_date: date("2024-01-15"),
  expiration_date: date("2044-01-15"),
  
  // Life specific fields
  cash_value: 0.00,
  beneficiaries: ["Jennifer Thompson (Spouse) - 100%"],
  medical_exam_date: date("2023-12-20"),
  medical_exam_results: "Approved - Standard",
  
  // Status
  policy_status: "Active",
  premium_status: "Current",
  last_payment_date: date("2024-01-15"),
  next_payment_due: date("2025-01-15"),
  
  created_at: datetime(),
  created_by: "underwriting",
  version: 1
})

// Create Relationships
CREATE (mike)-[:HOLDS_POLICY {
  start_date: term_policy.effective_date,
  policy_role: "Owner/Insured"
}]->(term_policy)

CREATE (term_policy)-[:BASED_ON {
  underwriting_date: term_policy.effective_date,
  risk_assessment: "Standard - Non-smoker"
}]->(term_life)

RETURN mike.full_name AS customer,
       term_policy.policy_number AS policy,
       term_policy.face_value AS coverage,
       term_policy.annual_premium AS premium
"""

term_results = run_query(term_life_query)
print("\nTerm Life Insurance Created:")
for record in term_results:
    print(f"  Customer: {record['customer']}")
    print(f"  Policy: {record['policy']}")
    print(f"  Coverage: ${record['coverage']:,.2f}")
    print(f"  Annual Premium: ${record['premium']:,.2f}")

## Step 4: Create Whole Life Insurance with Cash Value

### Business Context: Whole Life Cash Value

**Cash Value Accumulation:**
- Portion of premium builds cash value after first 1-2 years
- Grows at guaranteed rate (typically 2-4% annually)
- Tax-deferred growth with no annual tax on increases
- Available for policy loans at low interest rates

**Surrender Value vs. Cash Value:**
- Cash Value: Total accumulated value in the policy
- Surrender Value: Amount received if policy canceled (cash value minus surrender charges)
- Surrender charges typically decline over 8-15 years

**Dividend Options:**
- Paid in Cash: Receive annual dividend as check
- Reduce Premium: Apply dividend to lower annual premium
- Paid-up Additions: Purchase additional insurance (most common)
- Accumulate: Leave on deposit earning interest

In [ ]:
# Create whole life insurance customer and policy
whole_life_query = """
// Create Whole Life Customer
CREATE (sarah:Customer:Person {
  id: randomUUID(),
  customer_id: "CUST-LIFE-002",
  first_name: "Sarah",
  last_name: "Chen",
  full_name: "Sarah Chen",
  date_of_birth: date("1978-08-22"),
  age: 46,
  gender: "Female",
  
  // Contact information
  email: "sarah.chen@email.com",
  phone: "972-555-0165",
  
  // Address
  street_address: "2934 Maple Ridge Lane",
  city: "Plano",
  state: "TX",
  postal_code: "75023",
  country: "USA",
  
  // Life insurance specific
  smoking_status: false,
  health_status: "Good",
  occupation: "Physician",
  annual_income: 285000.00,
  
  created_at: datetime(),
  created_by: "underwriting",
  version: 1
})

// Match the Whole Life product
MATCH (whole_life:Product {product_id: "PROD-LIFE-002"})

// Create Whole Life Policy
CREATE (whole_policy:Policy:Life:Active {
  id: randomUUID(),
  policy_number: "POL-WHOLE-002934",
  product_type: "Life",
  policy_type: "Whole Life",
  
  // Coverage details
  face_value: 750000.00,
  premium_mode: "Annual",
  annual_premium: 8950.00,
  
  // Policy dates
  effective_date: date("2019-03-01"),
  
  // Whole life specific fields
  cash_value: 42850.00,
  cash_surrender_value: 38950.00,
  dividend_option: "Paid-up Additions",
  annual_dividend: 1250.00,
  beneficiaries: ["David Chen (Spouse) - 60%", "Emma Chen (Child) - 20%", "Alex Chen (Child) - 20%"],
  
  // Status
  policy_status: "Active",
  premium_status: "Current",
  last_payment_date: date("2024-03-01"),
  next_payment_due: date("2025-03-01"),
  
  created_at: datetime(),
  created_by: "underwriting",
  version: 1
})

// Create Relationships
CREATE (sarah)-[:HOLDS_POLICY {
  start_date: whole_policy.effective_date,
  policy_role: "Owner/Insured"
}]->(whole_policy)

CREATE (whole_policy)-[:BASED_ON {
  underwriting_date: whole_policy.effective_date,
  risk_assessment: "Standard - Professional"
}]->(whole_life)

RETURN sarah.full_name AS customer,
       whole_policy.policy_number AS policy,
       whole_policy.face_value AS coverage,
       whole_policy.cash_value AS cash_value,
       whole_policy.annual_premium AS premium
"""

whole_results = run_query(whole_life_query)
print("\nWhole Life Insurance Created:")
for record in whole_results:
    print(f"  Customer: {record['customer']}")
    print(f"  Policy: {record['policy']}")
    print(f"  Death Benefit: ${record['coverage']:,.2f}")
    print(f"  Cash Value: ${record['cash_value']:,.2f}")
    print(f"  Annual Premium: ${record['premium']:,.2f}")

## Step 5: Create Beneficiary Management

### Business Context: Beneficiary Designation

**Beneficiary Types:**
- **Primary Beneficiary**: First in line to receive death benefit
- **Contingent Beneficiary**: Receives benefit if primary is deceased
- **Tertiary Beneficiary**: Third level backup beneficiary

**Beneficiary Categories:**
- **Revocable**: Policy owner can change without beneficiary consent (most common)
- **Irrevocable**: Requires beneficiary consent to make changes

**Distribution Methods:**
- Per Stirpes: If beneficiary deceased, share goes to their descendants
- Per Capita: Divides equally among surviving beneficiaries
- Specific Percentage: Explicitly stated percentage to each beneficiary

In [ ]:
# Create beneficiaries and link to policies
beneficiary_query = """
// Create Beneficiaries
CREATE (jennifer:Beneficiary:Person {
  id: randomUUID(),
  beneficiary_id: "BEN-001",
  first_name: "Jennifer",
  last_name: "Thompson",
  full_name: "Jennifer Thompson",
  relationship: "Spouse",
  beneficiary_type: "Primary",
  percentage: 100.00,
  
  // Contact information
  email: "jennifer.thompson@email.com",
  phone: "214-555-0181",
  date_of_birth: date("1987-07-10"),
  
  // Verification
  identity_verified: true,
  verification_date: date("2024-01-10"),
  verification_method: "Government ID",
  
  created_at: datetime(),
  created_by: "underwriting",
  version: 1
})

CREATE (david:Beneficiary:Person {
  id: randomUUID(),
  beneficiary_id: "BEN-002",
  first_name: "David",
  last_name: "Chen",
  full_name: "David Chen",
  relationship: "Spouse",
  beneficiary_type: "Primary",
  percentage: 60.00,
  
  // Contact information
  email: "david.chen@email.com",
  phone: "972-555-0166",
  date_of_birth: date("1975-12-15"),
  
  // Verification
  identity_verified: true,
  verification_date: date("2019-02-20"),
  verification_method: "Government ID",
  
  created_at: datetime(),
  created_by: "underwriting",
  version: 1
})

// Link beneficiaries to policies
MATCH (term_policy:Policy {policy_number: "POL-TERM-001847"})
MATCH (whole_policy:Policy {policy_number: "POL-WHOLE-002934"})

CREATE (term_policy)-[:HAS_BENEFICIARY {
  designation_date: term_policy.effective_date,
  percentage: 100.00,
  beneficiary_type: "Primary"
}]->(jennifer)

CREATE (whole_policy)-[:HAS_BENEFICIARY {
  designation_date: whole_policy.effective_date,
  percentage: 60.00,
  beneficiary_type: "Primary"
}]->(david)

RETURN jennifer.full_name AS term_beneficiary,
       david.full_name AS whole_beneficiary
"""

beneficiary_results = run_query(beneficiary_query)
print("\nBeneficiaries Created:")
for record in beneficiary_results:
    print(f"  Term Life Beneficiary: {record['term_beneficiary']} (100%)")
    print(f"  Whole Life Primary Beneficiary: {record['whole_beneficiary']} (60%)")

## Step 6: Life Insurance Verification Queries

Verify the life insurance integration with comprehensive queries.

In [ ]:
# Query 1: Complete life insurance portfolio overview
portfolio_query = """
MATCH (c:Customer)-[h:HOLDS_POLICY]->(p:Policy:Life)-[:BASED_ON]->(prod:Product:Life)
OPTIONAL MATCH (p)-[:HAS_BENEFICIARY]->(b:Beneficiary)
RETURN c.full_name AS customer,
       p.policy_number AS policy,
       p.policy_type AS type,
       p.face_value AS death_benefit,
       p.cash_value AS cash_value,
       p.annual_premium AS premium,
       collect(DISTINCT b.full_name) AS beneficiaries
ORDER BY p.face_value DESC
"""

portfolio_df = pd.DataFrame(run_query(portfolio_query))
print("\nLife Insurance Portfolio:")
print(portfolio_df.to_string(index=False))

In [ ]:
# Query 2: Cash value analysis for whole life policies
cash_value_query = """
MATCH (c:Customer)-[:HOLDS_POLICY]->(p:Policy:Life)
WHERE p.policy_type = "Whole Life"
RETURN c.full_name AS customer,
       p.policy_number AS policy,
       p.face_value AS death_benefit,
       p.cash_value AS cash_value,
       p.cash_surrender_value AS surrender_value,
       p.dividend_option AS dividend_option,
       p.annual_dividend AS annual_dividend,
       round(p.cash_value * 100.0 / p.face_value, 2) AS cash_value_percentage
"""

cash_value_df = pd.DataFrame(run_query(cash_value_query))
print("\nWhole Life Cash Value Analysis:")
print(cash_value_df.to_string(index=False))

In [ ]:
# Query 3: Beneficiary distribution analysis
beneficiary_query = """
MATCH (p:Policy:Life)-[r:HAS_BENEFICIARY]->(b:Beneficiary)
MATCH (c:Customer)-[:HOLDS_POLICY]->(p)
RETURN c.full_name AS policy_owner,
       p.policy_number AS policy,
       p.face_value AS death_benefit,
       b.full_name AS beneficiary,
       b.relationship AS relationship,
       r.percentage AS percentage,
       round(p.face_value * r.percentage / 100.0, 2) AS benefit_amount
ORDER BY p.policy_number, r.percentage DESC
"""

beneficiary_df = pd.DataFrame(run_query(beneficiary_query))
print("\nBeneficiary Distribution:")
print(beneficiary_df.to_string(index=False))

In [ ]:
# Query 4: Life insurance product performance metrics
product_metrics_query = """
MATCH (prod:Product:Life)<-[:BASED_ON]-(p:Policy:Life)
WITH prod,
     count(p) AS policy_count,
     sum(p.face_value) AS total_death_benefit,
     sum(p.annual_premium) AS total_premium,
     avg(p.annual_premium) AS avg_premium
RETURN prod.product_name AS product,
       prod.coverage_type AS type,
       policy_count,
       total_death_benefit,
       total_premium,
       round(avg_premium, 2) AS avg_premium
ORDER BY total_death_benefit DESC
"""

product_metrics_df = pd.DataFrame(run_query(product_metrics_query))
print("\nLife Insurance Product Metrics:")
print(product_metrics_df.to_string(index=False))

## Step 7: Summary

### What You've Accomplished:

- Created **Term Life Insurance** products with flexible term options (10-30 years)
- Created **Whole Life Insurance** products with cash value accumulation and dividend options
- Implemented comprehensive **beneficiary management** with verification and percentage allocation
- Built life insurance policies with complete underwriting and medical exam tracking
- Created cash value tracking for whole life policies with surrender value calculation
- Established relationships between customers, policies, products, and beneficiaries

### Key Multi-Line Insurance Concepts:

1. **Term vs. Permanent Insurance**: Understanding the fundamental difference between temporary coverage and lifetime protection
2. **Cash Value Mechanics**: How whole life builds tax-deferred wealth while providing death benefit protection
3. **Beneficiary Structures**: Complex distribution strategies with primary, contingent, and percentage allocations
4. **Underwriting Classifications**: Risk assessment based on health, smoking, occupation, and lifestyle factors

---

**Next:** Continue to notebook 02 for Commercial Insurance integration.